***Review Link Scraping***

In [ ]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import csv



############################################################

# online
# Movies of five consecutive years from 2020-2016
movies_link = ["https://www.imdb.com/list/ls071285764/","https://www.imdb.com/list/ls071285512/","https://www.imdb.com/list/ls058813655/","https://www.imdb.com/list/ls062476111/","https://www.imdb.com/list/ls058938704/"]

cur_dir = '/content/drive/My Drive/ArtificialIntelligence_project'
out_file = open( cur_dir + '/dataset/movies.csv', 'w', newline='')


#write index
words  = ['title', 'reviews_link']
writer = csv.writer(out_file)
writer.writerow(words)
out_file.close()

for movies in movies_link:
  u_client = uReq(movies) # open url in code/memory html
  html_file = u_client.read()
  u_client.close() # not needed anymore

  # list of all movies titles
  bsoup = soup(html_file, "html.parser") 
  lst = bsoup.findAll("h3", {"class": "lister-item-header"})

  cur_dir = '/content/drive/My Drive/ArtificialIntelligence_project'
  out_file = open( cur_dir + '/dataset/movies.csv', 'a', newline='')


  for item in lst:
      title_tag = item.find("a", href = True)
      title = title_tag.text.strip()
      link = 'https://www.imdb.com' + title_tag['href'].split('?')[0] + 'reviews?ref_=tt_urv'
      if not title or not link:
          continue
      
      words  = [title, link]
      writer = csv.writer(out_file)
      writer.writerow(words)
  out_file.close()

***This Code is used to simulate Browser***

In [ ]:
!sudo apt-get install curl software-properties-common # read about curl
!curl -sL https://deb.nodesource.com/setup_10.x | sudo -E bash -
!sudo apt-get install nodejs # read nodejs and npm

In [ ]:
!npm -v
!node -v

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver #read about
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium #read about

In [ ]:
# read how does this works
from selenium import webdriver
def save_screenshot(driver: webdriver.Chrome, path: str = '/content/dataset/screenshot.png'):
    # Ref: https://stackoverflow.com/a/52572919/
    original_size = driver.get_window_size() # what does it do ?
    required_width = driver.execute_script('return document.body.parentNode.scrollWidth')
    required_height = driver.execute_script('return document.body.parentNode.scrollHeight')
    driver.set_window_size(required_width, required_height)
    # driver.save_screenshot(path)  # has scrollbar
    driver.find_element_by_tag_name('body').screenshot(path)  # avoids scrollbar
    driver.set_window_size(original_size['width'], original_size['height'])

In [ ]:
''' 
Testing code for a movie's review

Redundant code
'''

import time
from selenium import webdriver

# prepare the option for the chrome driver
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

# browser = webdriver.Chrome(chrome_path)

# set options to be headless, ..

# what is ECMA ?
options = webdriver.ChromeOptions()
options.add_argument('--headless') # why headless
options.add_argument('--no-sandbox') # why ?
options.add_argument('--disable-dev-shm-usage') # why ?

# open it, go to a website, and get results
chrome = webdriver.Chrome('chromedriver',options=options)

# search_box = chrome.find_elements_by_xpath("//input[@name='q' and @type='text' ]")[0]
# search_box.send_keys("print('Hello World')")
chrome.get('https://www.imdb.com/title/tt4500922/reviews?ref_=tt_sa_3')

load_button = chrome.find_elements_by_xpath("//*[@id='load-more-trigger']")[0]
load_button.click()
time.sleep(2)
load_button.click()
time.sleep(2)

save_screenshot(chrome)
print(chrome.page_source)  # results

Now we have installed all necessary modules needed for simulation of a browser in program.

The need for comes in picture because static html page of a movie's review page have only 25 reviews, and it has a button 'Load More' to load more reviews on that page.

To load more reviews on the page, we need to click that button, that's why we need to simulate browser.

In [ ]:
from selenium import webdriver

from urllib.request import urlopen as uReq

# read BeautifulSoup https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup as soup
import csv
import time


In [ ]:
import csv
cur_dir='/content/drive/My Drive/ArtificialIntelligence_project'
out_file = open( cur_dir + '/dataset/reviews.csv', 'w', newline='', encoding="utf-8")
k = 1

head  = ['imdb_link', 'review_title', 'review_text', 'rating']
writer = csv.writer(out_file) # read about csv module in python https://www.geeksforgeeks.org/working-csv-files-python/
writer.writerow(head)

data_path = cur_dir + '/dataset/movies.csv'
movies_list = open(data_path).read().split('\n')
movies_list=[]
with open(data_path,'r') as rf:
  reader = csv.reader(rf, delimiter=',')
  for row in reader:
    movies_list.append(row[1])
############## prepare browser simulation #############################

# prepare the option for the chrome driver
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

# browser = webdriver.Chrome(chrome_path)

# set options to be headless, ..

# read about ChromeOptions https://chromedriver.chromium.org/capabilities
options = webdriver.ChromeOptions() 
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

# use we.get(url) to get a web page with js in wd
############## browser is in wd #############################

# ignore first row, as it has header only
for movie in movies_list[1:len(movies_list)-1]:
    reviews_link=movie
    print(reviews_link + "\n")

    wd.get(reviews_link)

    try:
        load_button = wd.find_elements_by_xpath("//*[@id='load-more-trigger']")[0]
        for i in range(2):
            load_button.click()
            time.sleep(3)
    except:
        print('Load Button Not Found!!')
        continue

    html_file = wd.page_source

    bsoup = soup(html_file, "html.parser")
    #bsoup = bsoup.encode("utf-8")
    lst = bsoup.findAll("div", {"class": "lister-item-content"})

    for item in lst:

        title_tag = item.find("a", {"class": "title"})
        rating_tag = item.find("span", {"class": "rating-other-user-rating"})
        review_tag = item.find('div', {'class': 'text show-more__control'})

        # if any field is not available ignore this review
        if not rating_tag or not review_tag or not title_tag:
            continue

        imdb_link = reviews_link
        review_title = title_tag.text.strip()
        review_text = review_tag.text.strip()
        rating = rating_tag.findChildren('span')[0].text.strip()

        # verify output
        print("\n\nReview page: " + imdb_link
            + "\nReview title: " + review_title
            + "\nReview text: " + review_text
            + "\nRating: " + rating)
        # break


        words = [imdb_link, review_title, review_text, rating]
        writer = csv.writer(out_file)
        writer.writerow(words)
        
        # break
        
    # waiting for few seconds, so that imdb does not 
    # recognise our system as bot and does not block our ip
    time.sleep(3)
    # break
    
        
out_file.close()



Finally store our data (review_title, review_text, rating) in dataset.csv



In [ ]:

import csv


cur_dir = '/content/drive/My Drive/ArtificialIntelligence_project'

reviews_data = open(cur_dir + '/dataset/reviews.csv', 'r', newline='', encoding="utf-8")
dataset = open(cur_dir + '/dataset/datasets.csv', 'w', newline='', encoding="utf-8")

reviews_writer = csv.writer(dataset)
reviews_reader = csv.reader(reviews_data, delimiter=',')

# write index
words  = ['review_title', 'review', 'rating']
reviews_writer.writerow(words)

# print("Dataset:")
k = 1

for line in reviews_reader:
    if k==1:
        k = k+1
        continue
    words = review_title, review_text, rating = line[1:4]
    reviews_writer.writerow(words)

    # print first 10 reviews, for verification
    # if k<10:
    #     print("\n\nReview title: " + review_title
    #           + "\nReview text: " + review_text
    #           + "\nRating: " + rating)
    # k=k+1

reviews_data.close()
dataset.close()